In [15]:
import numpy as np
import pandas as pd
import nibabel as nib 
import glob

In [9]:
df=pd.read_table('~/GIT/ENIGMA/data/Behaviour_Information_ALL_April7_2022_sorted.csv', header=0, delimiter=',')
sub_left=df['LESIONED_HEMISPHERE']==1 

(1068, 21)


In [150]:
# load SMATT
smatt_dir= '/smatt_all_motor_rois/smatt-template/'
smatt_files = glob.glob(smatt_dir + '*.nii');

smatts = [smatt.replace(smatt_dir, '').replace('-S-MATT.nii', '') for smatt in smatt_files]
smatts.remove('S-MATT.nii')
smatts.sort()


LM1_CST = nib.load(smatt_dir + 'Left-M1-S-MATT.nii').get_fdata()
LPMd_CST = nib.load(smatt_dir + 'Left-PMd-S-MATT.nii').get_fdata()
LPMv_CST = nib.load(smatt_dir + 'Left-PMv-S-MATT.nii').get_fdata()
LS1_CST = nib.load(smatt_dir + 'Left-S1-S-MATT.nii').get_fdata()
LSMA_CST = nib.load(smatt_dir + 'Left-SMA-S-MATT.nii').get_fdata()
LpreSMA_CST = nib.load(smatt_dir + 'Left-preSMA-S-MATT.nii').get_fdata()

RM1_CST = nib.load(smatt_dir + 'Right-M1-S-MATT.nii').get_fdata()
RPMd_CST = nib.load(smatt_dir + 'Right-PMd-S-MATT.nii').get_fdata()
RPMv_CST = nib.load(smatt_dir + 'Right-PMv-S-MATT.nii').get_fdata()
RS1_CST = nib.load(smatt_dir + 'Right-S1-S-MATT.nii').get_fdata()
RSMA_CST = nib.load(smatt_dir + 'Right-SMA-S-MATT.nii').get_fdata()
RpreSMA_CST = nib.load(smatt_dir + 'Right-preSMA-S-MATT.nii').get_fdata()

colnames = ['Left-M1','Left-PMd','Left-PMv','Left-S1','Left-SMA','Left-preSMA','Right-M1','Right-PMd','Right-PMv','Right-S1','Right-SMA','Right-preSMA']


In [268]:
lesion_folder = '/Users/emilyolafson/GIT/ENIGMA/data/lesionmasks/all_lesionmasks_2009tov6_usingSym_rename/'

# calculate lesion load for each subject and append to growing table
sub_vecs=np.empty(shape=(0, 12))
                  
for sub in range(0,len(df)):
    lesion = nib.load(lesion_folder+  df.loc[df.index[sub],'BIDS_ID']+'.nii.gz').get_fdata();
    
    LM1_CST_sub=np.sum(lesion*LM1_CST)
    LPMd_CST_sub=np.sum(lesion*LPMd_CST)
    LPMv_CST_sub=np.sum(lesion*LPMv_CST)
    LS1_CST_sub=np.sum(lesion*LS1_CST)
    LSMA_CST_sub=np.sum(lesion*LSMA_CST)
    LpreSMA_CST_sub=np.sum(lesion*LpreSMA_CST)

    RM1_CST_sub=np.sum(lesion*RM1_CST)
    RPMd_CST_sub=np.sum(lesion*RPMd_CST)
    RPMv_CST_sub=np.sum(lesion*RPMv_CST)
    RS1_CST_sub=np.sum(lesion*RS1_CST)
    RSMA_CST_sub=np.sum(lesion*RSMA_CST)
    RpreSMA_CST_sub=np.sum(lesion*RpreSMA_CST)
    
    subvec=[LM1_CST_sub, LPMd_CST_sub, LPMv_CST_sub, LS1_CST_sub, LSMA_CST_sub, LpreSMA_CST_sub, RM1_CST_sub, RPMd_CST_sub, RPMv_CST_sub, RS1_CST_sub, RSMA_CST_sub, RpreSMA_CST_sub]
    sub_vecs=np.append(sub_vecs, np.reshape(subvec, [1, 12]), axis=0)

In [270]:
len(df)

1068

In [269]:
smatt_ll = pd.DataFrame(columns=smatts, data = sub_vecs)
print(smatt_ll)

      Left-M1  Left-PMd  Left-PMv  Left-S1  Left-SMA  Left-preSMA  Right-M1  \
0         0.0       0.0       0.0      0.0       0.0          0.0       0.0   
1         0.0       0.0       0.0      0.0       0.0          0.0       0.0   
2         0.0       0.0       0.0      0.0       0.0          0.0       0.0   
3         0.0       0.0       0.0      0.0       0.0          0.0       0.0   
4         0.0       0.0       0.0      0.0       0.0          0.0       0.0   
...       ...       ...       ...      ...       ...          ...       ...   
2131      0.0       0.0       0.0      0.0       0.0          0.0     306.0   
2132      0.0       0.0       0.0      0.0       0.0          0.0     386.0   
2133   1786.0     603.0    2224.0   1840.0     578.0        378.0       0.0   
2134      0.0       0.0       0.0      0.0       0.0          0.0       0.0   
2135     56.0      43.0      33.0     43.0      51.0         36.0      56.0   

      Right-PMd  Right-PMv  Right-S1  Right-SMA  Ri